# Sample Projects
This notebook is intended as a playground to try out various python widgets, ideas etc. This notebook is publically saved on github.

In [1]:
# tax rate is fixed
TAX_RATE = .0605

In [2]:
from math import floor

def round(x):
    return floor((x * 100) + .5)/100.0

class LineItem:
    def __init__(self,label,each_amount,qty):
        self.label = label
        self.each_amount = each_amount
        self.qty = qty
        self.line_amount = round(self.each_amount * self.qty)
    def __str__(self):
        if self.label == "LTAX":
            return "{} {} x {:0,.4f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)
        return "{} {} x ${:0,.2f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)

def invoice_str(invoice):
    return [str(i) for i in invoice]

In [3]:
def simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate):
    predicted_nightly_rate = round((gross_paid - tax_paid - cln_fee - extra_fee - (pet_fee * npets)) / stay_nights)
    rent = (predicted_nightly_rate * stay_nights)
    pets = pet_fee * npets
    subtotal = rent + cln_fee + pets + extra_fee
    tax_calc = round(subtotal * rate)
    total = subtotal + tax_calc
    invoice = [
        LineItem("RNT",predicted_nightly_rate,stay_nights),
        LineItem("CLN",cln_fee,1),
        LineItem("PET",pet_fee,npets),
        LineItem("XFEE",extra_fee,1),
        LineItem("LTAX",rate,subtotal)]
    print("Trying invoice={}".format(invoice_str(invoice)))
    
    if total != gross_paid:
        print("Invoice FAILED!!")
        raise RuntimeError("Could not predict invoice!")
        
    print("Invoice succeeded.")
    return invoice

In [4]:
def predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee=135.00,pet_fee=95.00,extra_fee=0.0, rate=.05):
    """
    Predicts the invoice line items given stay nights, gross amount paid,
    taxes paid, cleaning fee, and pet fees.  If the invoice can not be predicted,
    raise an exception.
    
    Parameters
    ----------
      stay_nights : integer
          The number of nights stayed. Used to multiple the nightly rate to determine overal rent.
      gross_paid : double
          The amount (USD) paid for the stay
      tax_paid : double
          The amount (USD) paid in taxes.  This should be total amount (rent + fees) * TAX_RATE.
      cln_fee : double
          The amount (USD) charged/paid for cleaning fees.
      pet_fee : double
          The amount (USD) charged/paid for pet fees.
    
    Returns
    -------
       A list of invoice items or raises an exception if the invoice could not be created
    """
    
#    # initialize the list of predictions
#    # a prediction is just a function that tries to return a list of invoice items
#    predictions = [simple_prediction]

    print("Predicting invoice:")
    print("   (stay_nights :: gross_paid :: tax_paid) = ({} :: ${:0,.2f} :: ${:0,.2f})".format(stay_nights,gross_paid,tax_paid))
#    prediction = predictions.pop()
    invoice = []
    npets = 0
    while True:
        try:
#            invoice = prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets)
            invoice = simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate)
            break
        except RuntimeError:
#            try:
#                prediction = predictions.pop()
#            except IndexError:
#                raise RuntimeError("Could not predict invoice!")
            npets += 1
            if npets==3 and rate==.0605:
                raise RuntimeError("Could not predict invoice!")
            elif npets==3:
                return predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,extra_fee,rate=.0605)

    return invoice

In [5]:
import csv

with open('payments.tsv') as payments_file:
    values_tsv = csv.reader(payments_file, delimiter='\t')
    data=[row for row in values_tsv]
    data=data[1:]
data

[['HA-F98TGY', '20210109', '5', '1178.1', '37.69', '1140.41', '56.1', '0'],
 ['HA-S1VXG5', '20210112', '4', '752.85', '24.09', '728.76', '35.85', '0'],
 ['HA-P3JL2X', '20210118', '4', '855.75', '27.38', '828.37', '40.75', '0'],
 ['HA-B2G3YJ', '20210124', '4', '935.55', '29.93', '905.62', '44.55', '0'],
 ['HA-0JTP0P', '20210129', '5', '924', '29.56', '894.44', '44', '0'],
 ['HA-3XYFRD', '20210130', '7', '1097.25', '35.11', '1062.14', '52.25', '0'],
 ['HA-1R496K', '20210213', '14', '3213.32', '102.82', '3110.5', '183.32', '0']]

In [6]:
def invoice_label(row):
    return "{} co {}:{}".format(row[0],row[1],row[3])

def to_invoice(row):
    try:
        return invoice_str(predict_invoice(stay_nights=int(row[2]),
                                          gross_paid=float(row[3]),
                                          tax_paid=float(row[6]),
                                          extra_fee=float(row[7])))
    except:
        return "INVOICE FAIL"

results = {invoice_label(row):to_invoice(row) for row in data}
results

Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (5 :: $1,178.10 :: $56.10)
Trying invoice=['RNT 5 x $197.40 = $987.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1122.0 x 0.0500 = $56.10']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $752.85 :: $35.85)
Trying invoice=['RNT 4 x $145.50 = $582.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 717.0 x 0.0500 = $35.85']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $855.75 :: $40.75)
Trying invoice=['RNT 4 x $170.00 = $680.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 815.0 x 0.0500 = $40.75']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $935.55 :: $44.55)
Trying invoice=['RNT 4 x $189.00 = $756.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 

{'HA-F98TGY co 20210109:1178.1': ['RNT 5 x $197.40 = $987.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1122.0 x 0.0500 = $56.10'],
 'HA-S1VXG5 co 20210112:752.85': ['RNT 4 x $145.50 = $582.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 717.0 x 0.0500 = $35.85'],
 'HA-P3JL2X co 20210118:855.75': ['RNT 4 x $170.00 = $680.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 815.0 x 0.0500 = $40.75'],
 'HA-B2G3YJ co 20210124:935.55': ['RNT 4 x $189.00 = $756.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 891.0 x 0.0500 = $44.55'],
 'HA-0JTP0P co 20210129:924': ['RNT 5 x $149.00 = $745.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 880.0 x 0.0500 = $44.00'],
 'HA-3XYFRD co 20210130:1097.25': ['RNT 7 x $130.00 = $910.00',
  'CLN 1 x $1